# Исследование надежности заемщиков

## Шаг 1. Обзор данных

Составление первого представления о данных статистики о платежеспособности клиентов.

In [1]:
# импорт библиотеки pandas
import pandas as pd

Чтение файла data.csv и сохрание его в переменной df:

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

Вывод на экран первых десяти строк таблицы:

In [3]:
# получение первых 10 строк таблицы df
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о таблице:

In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. Тип данных во всех столбцах разный и соответствует содержимому этих столбцов.

Согласно документации к данным:

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

В названиях столбцов не видно никаких нарушений стиля.

А вот в столбце с данными об образовании `education` строчные буквы сочетаются с прописными. В некоторых столбцах, например в `days_employed` значения лучше заменить на целочисленные, а также выяснить причину появление отрицательных значений и исправить их.

Количество значений в столбцах различается. Значит, в данных двух стобцов (`days_employed` и `total_income`) имеются пропущенные значения.

## Шаг 2. Предобработка данных

### Заполнение пропусков

В двух столбцах (`days_employed` и `total_income`) есть пропущенные значения, необходимо найти их:

In [5]:
# подсчёт пропусков
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# расчет доли пропусков в процентах от выборки: количество пропусков / общее количество строк
miss_part = 2174 / 21525
print(f'Доля пропусков составляет {miss_part:.0%}')

Доля пропусков составляет 10%


* Общий трудовой стаж в днях и ежемесячный доход — количественные переменные. Пропуски в таких переменных заполняют характерными значениями. Чтобы примерно оценить типичные значения выборки, используется среднее арифметическое или медиана.
* Количество пропущеных значений в обоих столбцах одинаково и составляет 10% от выборки. 
* Пропуски могли появиться по технологическим причинам при перезаписи или переносе данных, либо из-за человеческого фактора. Возможно, по каким-то причинам данные не были предоставлены или были утеряны, хотя первое маловероятно при оформлении кредита.
* Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства, что характерно для общего трудового стажа в днях и ежемесячного дохода. Поэтому медиана даст более объективную информацию при заполнении пропусков значений в этих столбцах, чем среднее значение. 

Пропуски необходимо заполнить медианным значением по столбцу:

In [7]:
# подсчет медианы для столбца `days_employed`
days_employed_avg = df['days_employed'].median()

# замена пропущенных значений в столбце `days_employed` на медианное значение этого столбца
df['days_employed'] = df['days_employed'].fillna(value=days_employed_avg) 

# подсчет медианы для столбца `total_income`
total_income_avg = df['total_income'].median()

# замена пропущенных значений в столбце `total_income` на медианное значение этого столбца
df['total_income'] = df['total_income'].fillna(value=total_income_avg)

Проверка, что в таблице не осталось пропусков. Для этого ещё раз произведен подсчет пропущенных значений.

In [8]:
# подсчёт пропусков
display(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления

In [9]:
# функция для просмотра уникальных значений
def uni(column):
    return display(df[column].unique())

Просмотр уникальных значений для столбца `children`, с целью выявления аномальных данных:

In [10]:
# просмотр уникальных значений в столбце `children`
uni('children')

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` присутствуют такие значения как: -1 и 20, они, вероятнее всего, являются некорректными, возможно из-за опечатки при переносе данных.
Определение количества повторений этих значений в данном столбце:

In [11]:
# подсчет колличества повторений уникальных значений в столбце `children`
display(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Можно предположить, что -1 это 1, а 20 это 2, но предположение может оказаться неверным и повлиять на дальнейшее исследовние, проверка доли значений -1 и 20 для возможности дальнейшего удаления этих значений:

In [53]:
# расчет доли значения -1 в процентах от выборки: количество значений / общее количество строк
part_m1 = 47 / len(df['children'])
print(f'Доля значения -1 составляет {part_m1:.2%}')

# расчет доли значения 20 в процентах от выборки: количество значений / общее количество строк
part_20 = 76 / len(df['children'])
print(f'Доля значения 20 составляет {part_20:.2%}')

Доля значения -1 составляет 0.22%
Доля значения 20 составляет 0.35%


Доля значений -1 и 20 достаточно мала (менее 5%), значит удаление этих значений не окажет влияния на дальнейшее исследование.

Удаление значений -1 и 20 из столбца `children`:

In [56]:
# фильтрация значений, для избавления от значения -1
df = df[df['children'] != -1]

# фильтрация значений, для избавления от значения 20
df = df[df['children'] != 20]

Проверка, что в столбце не осталось некорректных значений:

In [57]:
# проверка
display(df['children'].value_counts())

0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

В столбце `days_employed` присутствуют отрицательные и аномально большие значения количества дней трудового стажа. Вероятно данные, отчасти, записаны в часах. Причиной такой записи может быть то, что в данных встречается информация из двух систем, одна из которых записывает стаж в часах.

Предположим, что в среднем человек работает 44,5 года, исходя из:

In [58]:
# расчет среднего возраста выхода на пенсию: исходя из 60 лет для женщин и 65 лет для мужчин
work_end_avg = (60 + 65)/2

# среднего возраста выхода на пенсию минус 18, возраст начала работы
employed_avg = work_end_avg - 18
print(employed_avg)

44.5


In [59]:
# расчет среднего количества рабочих дней
days_employed_avg = employed_avg * 365
print(days_employed_avg)

16242.5


Таким образом получено среднее количество дней рабочего стажа.
Исправление отрицательных и аномально больших значений:

In [60]:
# взятие значения по модулю
df['days_employed'] = df['days_employed'].apply(abs)

# расчет
df.loc[df['days_employed'] > days_employed_avg, 'days_employed'] = (df.loc[df['days_employed'] > days_employed_avg, 
                                                                           'days_employed'] / 24)

# проверка
display(df['days_employed'].head())

0     8437.0
1     4024.0
2     5623.0
3     4124.0
4    14177.0
Name: days_employed, dtype: float64

В столбце `days_employed` больше нет отрицательных и аномально больших значений.

Просмотр уникальных значений для столбца `dob_years`, с целью выявления аномальных данных:

In [61]:
# просмотр уникальных значений в столбце `dob_years`
display(df['dob_years'].value_counts())

42    692
35    614
41    603
40    603
34    597
38    595
33    577
39    572
31    556
36    553
44    543
29    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    262
23    252
65    194
22    183
66    183
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце `dob_years` присутствует значение 0, оно является некорректными, возможно из-за опечатки при переносе данных.
Замена значения на медианное:

In [62]:
# расчет медианы для столбца `dob_years`
dob_years_median = df['dob_years'].median()

# замена нулевых значений на медианное
df.loc[df['dob_years'] == 0, 'dob_years'] = dob_years_median

# проверка
display(df['dob_years'].value_counts())

42.0    692
35.0    614
41.0    603
40.0    603
34.0    597
38.0    595
33.0    577
39.0    572
31.0    556
36.0    553
29.0    543
44.0    543
48.0    536
30.0    536
37.0    531
43.0    510
50.0    509
32.0    506
49.0    505
28.0    501
45.0    494
27.0    490
52.0    483
56.0    482
47.0    480
54.0    476
46.0    469
58.0    461
53.0    457
57.0    457
51.0    446
59.0    441
55.0    441
26.0    406
60.0    376
25.0    356
61.0    353
62.0    351
63.0    268
64.0    263
24.0    262
23.0    252
65.0    194
66.0    183
22.0    183
67.0    167
21.0    110
68.0     99
69.0     83
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

Теперь в столбце нет клиентов с нулевым возрастом.

Просмотр уникальных значений для столбца `education_id`, с целью выявления аномальных данных:

In [63]:
# просмотр уникальных названий в столбце `education_id`
uni('education_id')

array([0, 1, 2, 3, 4])

Аномальных значений в столбце `education_id` - нет.

Просмотр уникальных значений для столбца `family_status`, с целью выявления аномальных данных:

In [64]:
# просмотр уникальных названий в столбце `family_status`
uni('family_status')

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Аномальных значений в столбце `family_status` - нет. Не смотря на то, что 'Не женат / не замужем', указано с большой буквы, невных дубликатов нет, поэтому данные этого столбца будут оставлены в таком виде.

Просмотр уникальных значений для столбца `family_status_id`, с целью выявления аномальных данных:

In [65]:
# просмотр уникальных названий в столбце `family_status_id`
uni('family_status_id')

array([0, 1, 2, 3, 4])

Аномальных значений в столбце `family_status_id` - нет.

Просмотр уникальных значений для столбца `gender`, с целью выявления аномальных данных:

In [66]:
# просмотр уникальных названий в столбце `gender`
uni('gender')

array(['F', 'M'], dtype=object)

В столбце `gender` присутствует значения XNA, вероятнее всего данные отсутствовали. Определение количества повторений этого значения в данном столбце:

In [67]:
# подсчет повторений уникальных значений
display(df['gender'].value_counts())

F    14154
M     7247
Name: gender, dtype: int64

Значение XNA повторяется в столбце всего лишь один раз, это очень маленькая доля от всей выборки, значит строку с этим значением можно просто убрать:

In [69]:
# фильтрация значений, для избавления от значения XNA
df = df[df['gender'] != 'XNA']

# проверка
display(df['gender'].value_counts())

F    14154
M     7247
Name: gender, dtype: int64

Теперь в столбце `gender` у всех есть пол, а строки с неопределенным полом больше нет.

Просмотр уникальных значений для столбцов `income_type`, `debt` и `purpose`, с целью выявления аномальных данных:

In [70]:
# просмотр уникальных названий в столбце `income_type`
uni('income_type')

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [71]:
# просмотр уникальных названий в столбце `debt`
uni('debt') 

array([0, 1])

In [72]:
# просмотр уникальных названий в столбце `purpose`
uni('purpose')

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Аномальных значений в столбцах `income_type`, `debt` и `purpose` - нет.

### Изменение типов данных

При замене нулевых значений на медианное в столбце `dob_years` тип данных изменился на float, так как медианное значение было типа float.
Изменение типа данных в столбце на int:

In [73]:
# изменение типа данных с float на int
df['dob_years'] = df['dob_years'].astype(int)

# проверка 
display(df['dob_years'].value_counts().head())

42    692
35    614
41    603
40    603
34    597
Name: dob_years, dtype: int64

В столбце `total_income` значения представленны в виде вещественного типа данных.
Произведена замена на целочисленный:

In [74]:
# изменение типа данных с float на int
df['total_income'] = df['total_income'].astype(int)

# проверка 
display(df['total_income'].head())

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

Аналогично и для столбца `days_employed` , для удобства восприятия.

In [75]:
# изменение типа данных с float на int
df['days_employed'] = df['days_employed'].astype(int)

# проверка 
display(df['days_employed'].head())

0     8437
1     4024
2     5623
3     4124
4    14177
Name: days_employed, dtype: int64

### Удаление дубликатов

Поиск неявных дубликатов:

In [76]:
# просмотр уникальных значений в столбце `education`
uni('education')

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

В столбце education в названиях строчные буквы сочетаются с прописными, из-за этого происходит некорректный подсчет уникальных значений. Данные дубликаты могли возникнуть вследствие переноса данных или при формировании данных из-за человеческого фактора. Приведение букв к одному регистру:

In [77]:
# приведение к общему, нижнему, регистру
df['education'] = df['education'].str.lower()

# проверка, что в столбце не осталось некорректных значений
uni('education')

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Так как проблема состояла только в том, что использовался разный регистр, то наиболее оптимальный метод для исправления дубликатов в `education` это - приведение к общему регистру с помощью str.lower().

Явные дубликаты:

In [78]:
# подсчет количества явных дупликатов
print(df.duplicated().sum())

72


In [80]:
# удаление явных дубликатов без пропуска в индексах
display(df.drop_duplicates().reset_index(drop=True))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21324,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21325,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21326,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21327,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


Явных дубликатов больше нет.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создание двух новых датафреймов:

In [81]:
# подготовка словаря
education_dict = df[['education_id','education']] 
display(education_dict)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
...,...,...
21520,1,среднее
21521,1,среднее
21522,1,среднее
21523,1,среднее


In [82]:
# группировка исходного датафрейма
debt_pivot_edu = df.pivot_table(index = 'education_id', columns = 'debt', values = 'education', aggfunc = 'count').fillna(0)
display(debt_pivot_edu)

debt,0,1
education_id,,
0,4959.0,278.0
1,13781.0,1355.0
2,672.0,68.0
3,251.0,31.0
4,6.0,0.0


In [83]:
# соединение словаря и группировки
education_m = education_dict.merge(debt_pivot_edu, on = 'education_id')
display(education_m)

,education_id,education,0,1
0,0,высшее,4959.0,278.0
1,0,высшее,4959.0,278.0
2,0,высшее,4959.0,278.0
3,0,высшее,4959.0,278.0
4,0,высшее,4959.0,278.0
...,...,...,...,...
21396,4,ученая степень,6.0,0.0
21397,4,ученая степень,6.0,0.0
21398,4,ученая степень,6.0,0.0
21399,4,ученая степень,6.0,0.0


In [84]:
# подготовка словаря
family_status_dict = df[['family_status_id','family_status']]
display(family_status_dict)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
...,...,...
21520,1,гражданский брак
21521,0,женат / замужем
21522,1,гражданский брак
21523,0,женат / замужем


In [85]:
# группировка исходного датафрейма
debt_pivot_famil = df.pivot_table(index = 'family_status_id', columns = 'debt', values = 'family_status', aggfunc = 'count')
display(debt_pivot_famil)

debt,0,1
family_status_id,,
0,11375,927
1,3774,385
2,889,63
3,1105,84
4,2526,273


In [86]:
# соединение словаря и группировки
family_status_m = family_status_dict.merge(debt_pivot_famil, on = 'family_status_id')
display(family_status_m)

,family_status_id,family_status,0,1
0,0,женат / замужем,11375,927
1,0,женат / замужем,11375,927
2,0,женат / замужем,11375,927
3,0,женат / замужем,11375,927
4,0,женат / замужем,11375,927
...,...,...,...,...
21396,4,Не женат / не замужем,2526,273
21397,4,Не женат / не замужем,2526,273
21398,4,Не женат / не замужем,2526,273
21399,4,Не женат / не замужем,2526,273


Удаление столбцов `education` и `family_status` из исходного датафрейма:

In [88]:
# для удобства дальнейших категоризаций
df_new = df.drop(['education', 'family_status'], axis=1) 
display(df_new.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Категоризация дохода

In [89]:
# функция для категоризации
def income_category(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

# применение функции с записью в новый столбец `total_income_category`
df_new['total_income_category'] = df_new['total_income'].apply(income_category)
display(df_new.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [90]:
# просмотр уникальных названий в столбце `purpose`
display(df_new['purpose'].value_counts())

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

In [91]:
# функция для категоризации
def purpose_categories(purposes):
    purpose_dict = {'проведение свадьбы': ['свадьба', 'свадьбы', 'свадьбу'],
                    'операции с недвижимостью': ['жилья', 'жильем', 'жилью' 'недвижимостью', 'недвижимости', 'недвижимость'],
                    'операции с автомобилем': ['автомобиля','автомобиль','автомобилем','автомобили'],
                    'получение образования': ['образованием', 'образование', 'образования']} # создание словаря синонимов
    for cat, synonymns in purpose_dict.items(): # перебор синонимов
        for syn in synonymns:
            if syn in purposes: return cat
    return None

# применение функции с записью в новый столбец `purpose_category`
df_new['purpose_category'] = df_new['purpose'].apply(purpose_categories)
display(df_new.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Шаг 3. Ответы на вопросы

### Проверка гипотезы: Количество детей оказывает влияние на возврат кредита в срок.

In [92]:
# создание сводной таблицы
report_children = df_new.pivot_table(index = 'children', values = 'debt', aggfunc = ['sum', 'count', 'mean'])

# названия колонок
report_children.columns = ['debt', 'total', '%']

# сортировка
report_children = report_children.sort_values(by = ['%'], ascending = False)
display(report_children)

,debt,total,%
children,,,
4,4,41,0.097561
2,194,2055,0.094404
1,444,4818,0.092154
3,27,330,0.081818
0,1063,14148,0.075134
5,0,9,0.000000


**Вывод:**

Исходя из результата, количество детей не влияет на выплату кредита в срок. Разница между теми, у кого пятеро детей и теми, у кого их нет, присутствует, но незначительна, семьи с пятью детьми не имеют задолженностей, но таковых семей намного меньше, что тоже важно учитывать. Гипотеза о том, что количество детей оказывает влияние на возврат кредита в срок не подтверждается.

### Проверка гипотезы: Семейное положение оказывает влияние на возврат кредита в срок.

In [93]:
# создание сводной таблицы
report_family = df.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['sum', 'count', 'mean'])

# названия колонок
report_family.columns = ['debt', 'total', '%']

# сортировка
report_family = report_family.sort_values(by = ['%'], ascending = False)
display(report_family)

,debt,total,%
family_status,,,
Не женат / не замужем,273,2799,0.097535
гражданский брак,385,4159,0.092570
женат / замужем,927,12302,0.075354
в разводе,84,1189,0.070648
вдовец / вдова,63,952,0.066176


**Вывод:**

Семейное положение не оказывает влияние на возврат кредита в срок, эта гипотеза не подтверждается. Разница между людьми категории "вдовец / вдова" и категории "Не женат / не замужем" присутствует, но также незначительная.

### Проверка гипотезы: Уровень дохода оказывает влияние на возврат кредита в срок.

In [94]:
# создание сводной таблицы
report_income = df_new.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['sum', 'count', 'mean']) 

# названия колонок
report_income.columns = ['debt', 'total', '%']

# сортировка
report_income = report_income.sort_values(by = ['total_income_category'], ascending = False)
display(report_income)

,debt,total,%
total_income_category,,,
E,2,22,0.090909
D,21,349,0.060172
C,1353,15993,0.084600
B,354,5012,0.070630
A,2,25,0.080000


**Вывод:**

Уровень дохода оказывает влияние на возврат кредита в срок - не подтверждается. Люди с меньшим доходом имеют больше задолженностей по кредитам, как и люди с высоким доходом, вероятно это связано с тем, что таких людей не очень мало, а вот людей со средним, стабильным уровнем дохода достаточное количество и уровень своевременных выплат кредита чуть выше, но не значительно.

### Проверка гипотезы: Цели взятия кредита оказывают влияние на возврат его в срок.

In [95]:
# создание сводной таблицы
report_purpose = df_new.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['sum', 'count', 'mean'])

# названия колонок
report_purpose.columns = ['debt', 'total', '%']

# сортировка
report_purpose = report_purpose.sort_values(by = ['total'], ascending = False)
display(report_purpose)

,debt,total,%
purpose_category,,,
операции с недвижимостью,699,9528,0.073363
операции с автомобилем,400,4288,0.093284
получение образования,369,3997,0.092319
проведение свадьбы,183,2337,0.078306


**Вывод:**

Цели взятия кредита оказывают влияние на возврат его в срок - не подтверждается. Разница между взятием кредита на недвижимость, свадьбу незназначительно отличается от взятия кредита на автомобиль или образование. 

## Общий вывод

In [97]:
# создание сводной таблицы
report = df_new.pivot_table(index = ['children','days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender',
                                     'income_type', 'total_income', 'purpose_category'], values = 'debt', 
                            aggfunc = ['sum', 'count', 'mean'])

# названия колонок
report.columns = ['debt', 'total', '%']

# сортировка
report = report.sort_values(by = ['%'], ascending = True)
report_pivot_with_reset_index = report.reset_index()
display(report_pivot_with_reset_index.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,total_income,purpose_category,debt,total,%
0,0,24,32,0,4,M,сотрудник,124115,получение образования,0,1,0.0
1,0,16019,65,1,2,F,пенсионер,198209,получение образования,0,1,0.0
2,0,16017,56,0,0,M,пенсионер,160832,операции с недвижимостью,0,1,0.0
3,0,16015,64,1,1,F,пенсионер,90782,проведение свадьбы,0,1,0.0
4,0,16014,60,1,4,F,пенсионер,97511,операции с автомобилем,0,1,0.0
5,0,16013,58,1,4,F,пенсионер,52030,операции с недвижимостью,0,1,0.0
6,0,16011,63,1,0,F,пенсионер,71986,операции с недвижимостью,0,1,0.0
7,0,16020,54,1,1,F,пенсионер,147562,проведение свадьбы,0,1,0.0
8,0,16009,61,1,0,M,пенсионер,203444,операции с автомобилем,0,1,0.0
9,0,16008,51,1,0,F,пенсионер,54142,операции с автомобилем,0,1,0.0


Если учитывать максимальное количество факторов (Количество детей, общий трудовой стаж в днях, возраст клиента в годах, образование, семейное положение, пол, типа занятости, доход, цель получения кредита), то самым надежным заемщиком будет являться: Человек не имеющий детей, с трудовым стажем в 24 дня, в возрасте 32 лет, высшим образованием, в семейном положение Не женат / не замужем, мужского пола, сотрудник, с доходом 124115 и целью взятия кредита на получение образования.